In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

import numpy as np
import re

from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import scienceplots

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg
from COMBINE_harmonizer import utils_corr

plt.style.use('nature')
plt.rcParams['legend.frameon'] = False
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

## 01. Init

In [2]:
root_dir = '../'

In [3]:
COMBINE_harmonizer.init(f'{root_dir}/config.yaml')

In [4]:
input_dir = f"{cfg.config['out_dir']}/stats"
out_dir = f"{cfg.config['out_dir']}/stats"

out_figure_dir = f"{out_dir}/figure"

os.makedirs(out_figure_dir, exist_ok=True)

data_dict_filename = f"{root_dir}/{COMBINE_harmonizer.DATA_DICTIONARY_EXCEL}"

In [5]:
df_data_dict_main = COMBINE_harmonizer.load_data_dict(data_dict_filename, COMBINE_harmonizer.SHEET_MAIN)
df_data_dict_followup = COMBINE_harmonizer.load_data_dict(data_dict_filename, COMBINE_harmonizer.SHEET_FOLLOW_UP)

data_dict_main_map = {row[COMBINE_harmonizer.DATA_DICT_VAR_NAME]: row[COMBINE_harmonizer.DATA_DICT_VAR_TYPE] for idx, row in df_data_dict_main.iterrows()}
data_dict_followup_map = {row[COMBINE_harmonizer.DATA_DICT_VAR_NAME]: row[COMBINE_harmonizer.DATA_DICT_VAR_TYPE] for idx, row in df_data_dict_followup.iterrows()}


_DATA_DICT_MAP = {
    COMBINE_harmonizer.SHEET_MAIN: data_dict_main_map,
    COMBINE_harmonizer.SHEET_FOLLOW_UP: data_dict_followup_map,
}

In [6]:

_FILENAME_INFO_DATA_DICT_MAP = {each['name']: _DATA_DICT_MAP[each['data_dict']] for each in COMBINE_harmonizer.FILENAME_INFOS}

_PREFIX_DATA_DICT_MAP = {COMBINE_harmonizer.flatten_filename_prefix(filename): val for filename, val in _FILENAME_INFO_DATA_DICT_MAP.items()}

## Load flatten

In [7]:
filename = f'{input_dir}/zz-merged-flatten-numeric.csv'

In [8]:
df_numeric = pd.read_csv(filename)

/var/folders/84/vxdpc5mx2cqdr6zvp8p_tygc0000gn/T/ipykernel_79367/2373133381.py:1: DtypeWarning: Columns (6,40,44,45,161,163,166,198,201,207,210,212,216,217,220,228,229,230,231,236,240,241,242,243,248,252,253,254,255,256,260,262,2041,2042,2043,2044,2045,2869,2870,2871,2872,2873,2874,2875,2876,2877,2878,2879,2880,2881,2882,2883,2884,2885,2886,2887,2888,2889,2890,2891,2892,2893,2894,2895,2896,2897,2898,2899,2900,2901,2902,2903,2904,2905,2906,2907,2908,2909,2910,3273,3277,3280,3283,3284,3286,3287,3288,3290,3301,3302,3303,3311,3312,3316,3317,3318,3319,3321,3326,3327,3328,3330,3341,3342,3343,3346,3348,3349,3350,3351,3352,3353,3356,3357,3358,3359,3360,3376,3377,3378,3379,3466,3470,3471,3472,3478,3498,3499,3500,3501,3502,3503,3504,3505,3506,3510,3511,3512,3518,3524,3525,3758,3759,3760,3761,3762,3763,3764,3765,3766,3767,3768,3769,3770,3771,3772,3773,3774,3775,3776,3777,3778,3779,3780,3781,3782,3783,3784,3785,3786,3787,3788,3789,3793,3794,3795,3796,3797,3799,3801,3802,3803,3804,3805,3806,3807,38

In [9]:
columns = list(df_numeric.columns)

In [10]:
df_columns = pd.DataFrame({'columns': columns})

In [11]:
df_columns['column_tuple'] = df_columns['columns'].apply(COMBINE_harmonizer.flatten_column_tuple)
df_columns['column_name'] = df_columns['column_tuple'].apply(lambda x: x[1])
df_columns['column_prefix'] = df_columns['column_tuple'].apply(lambda x: x[0])

In [12]:
df_columns['column_prefix'].unique()

array(['', '01-02', '01-03', '01-04', '01-05', '01-05_1', '01-05_2',
       '01-06', '01-07', '01-08', '01-09', '01-10', '01-11', '01-12',
       '01-12_1', '02-01', '02-02', '02-03', '02-04', '02-05', '02-05_s',
       '02-06_s', '02-07', '02-08', '02-09', '02-11', '02-12', '02-13',
       '02-14', '02-15', '02-16', '02-17', '03-01', '03-01_s', '03-02',
       '03-03', '03-04', '03-04_1', '03-05', '03-05_s', '03-05_s1',
       '04-01', '04-01_1', '04-02', '04-03', '04-04', '04-05', '04-06',
       '04-07', '04-08', '04-09', '04-10', '04-11', '04-12', '04-12_1',
       '04-13', '04-14', '04-15', '04-16', '04-17', '20-00', '20-01',
       '20-02', '20-03', '20-04', '20-05', '20-06', '20-07', '20-08',
       '20-09', '20-10', '20-10_1'], dtype=object)

In [13]:
columns = list(df_numeric.columns)
valid_columns = []
for idx, column in tqdm(enumerate(columns)):
    prefix, var_name, postfix = COMBINE_harmonizer.flatten_column_tuple(column)
    if prefix not in _PREFIX_DATA_DICT_MAP:
        print(f'[WARN] ({idx}/{len(columns)}) not in _PREFIX_DATA_DICT_MAP: column: {column} prefix: {prefix}')
        continue

    data_dict_map = _PREFIX_DATA_DICT_MAP[prefix]
    if var_name not in data_dict_map:
        print(f'[WARN] ({idx}/{len(columns)}) not in DATA_DICT_MAP: {var_name}')
        continue
    valid_columns.append(column)

df_numeric_valid = df_numeric[valid_columns]

0it [00:00, ?it/s]

6119it [00:00, 1276354.99it/s]

[WARN] (0/6119) not in _PREFIX_DATA_DICT_MAP: column: _study prefix: 
[WARN] (1/6119) not in _PREFIX_DATA_DICT_MAP: column: center prefix: 
[WARN] (2/6119) not in _PREFIX_DATA_DICT_MAP: column: subjectID prefix: 
[WARN] (3/6119) not in _PREFIX_DATA_DICT_MAP: column: uniqueID prefix: 
[WARN] (5357/6119) not in DATA_DICT_MAP: MRI_ID
[WARN] (5358/6119) not in DATA_DICT_MAP: MRI_ID
[WARN] (5359/6119) not in DATA_DICT_MAP: MRI_ID
[WARN] (5621/6119) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (5629/6119) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (5706/6119) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (5773/6119) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (5871/6119) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (5915/6119) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (5920/6119) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (6031/6119) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (6092/6119) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (6104/6119) not in DATA_DICT_MAP: uniqueFo

In [14]:
len(df_numeric_valid.columns), list(df_numeric_valid.columns)

(6102,
 ['01-02:siteID',
  '01-02:birthNumber',
  '01-02:screenComment',
  '01-02:coreTempLess32p5CGreaterEq2Hr_e',
  '01-02:coreTempLess33p5CGreater1Hr_e',
  '01-02:coreTempLess34CGreater1Hr_e',
  '01-02:first6HrCoolByClinicalProtocol_e',
  '01-02:chromosomalAbnormality_e',
  '01-02:majorCongenitalAnomaly_e',
  '01-02:birthWeightLessEq1800g_e',
  '01-02:infantUnlikelySurvive_e',
  '01-02:first60MinAllBloodGasPHGreater7p15BaseDeficitLess10mEqPerL_e',
  '01-02:postnatalAgeLess6HrOrGreater24Hr_e',
  '01-02:enrolledConflictingTrial_e',
  '01-02:first60MinAnyBloodGasPHLessEq7_i',
  '01-02:first60MinAnyBloodGasBaseDeficitGreaterEq16mEqPerL_i',
  '01-02:historyPerinatalEvent_i',
  '01-02:at10MinApgarLessEq5OrVent_i',
  '01-02:randomEligible',
  '01-02:consentStatus',
  '01-02:noConsentReason',
  '01-02:noInStudyReason',
  '01-02:random',
  '01-02:noRandomReason',
  '01-02:noRandomReasonText',
  '01-02:randomDate',
  '01-02:randomTime',
  '01-02:ageRand_hr',
  '01-02:randomNumber',
  '01-02:r

## 03. all

In [15]:
y_columns = [
    '20-10:moderateSevereDisabilityOrDeath',
    '20-10:disabilityLevelDeath4Category',
    '20-10_1:disabilityLevelDeath',

    '20-04:BayleyIIICognitiveComposite',
    '20-04:BayleyIIILanguageComposite',
    '20-04:BayleyIIIMotorComposite',
]

value_columns = [
    'corr',
    'r2',
    'pvalue',
    'valid',
    'valid_x',
    'valid_y',
    'valid_percent',
]

def _reorder_columns(y_columns, value_columns):
    for y_column in y_columns:
        for value_column in value_columns:
            yield y_column, value_column

reorder_columns = list(_reorder_columns(y_columns, value_columns))

In [16]:
_EXCLUDE_COLUMNS = y_columns + [
    '01-01:birthNumber', # 1: 525 2: 7
    '01-01:coreTempBelow32p5COver2H', # 0: 532 (fit criteria),
    '01-07:pre_CoolbyIceGelPack', # 0: 250, 1: 2

    '20-09:blindness', # part of the outcome definition
    '20-09:multipleImpairment', # part of the outcome defition
    '20-09:moderateSevereCerebralPalsy', # part of the outcome defition
    '20-09:hearingImpairedWithAid', # part of the outcome defition
    '20-09:cerebralPalsyMerge', # part of the outcome defition
    '20-09:afterDischargeSeizure', # part of the outcome defition
    '20-09:grossMotorFunctionLevelSeverity', # part of the outcome defition

    '04-01:deathAge_day', # related t the outcome definition

    '20-10:normalPrimaryOutcome', # part of the outcome defition
    '20-10:moderateSevereDisabilitySurvivor', # part of the outcome defition
    '20-10:disabilityLevelSurvivor', # part of the outcome defition
    '20-10:disabilityLevelDeath4Category', # part of the outcome defition
    '20-10:outcomeGroup', # part of the outcome defition
]

_INCLUDE_COLUMN_PREFIX = [
    '01-02',
    '01-03',
    '01-04',
    '01-05',
    '01-05_1',
    '01-05_2',
    '01-06',
    '01-07',
    '01-08',
    '01-09',
    '01-10',
    '01-12',
    '01-12_1',
    '03-01_s',
    '03-02',
    '03-04',
    '03-04_1',
    '03-05',
    '03-05_s',
    '03-05_s1',
    '04-01',
    '04-01_1',
    '04-02',
    '04-03',
    '04-04',
    '04-05',
    '04-06',
    '04-07',
    '04-08',
    '04-09',
    '04-10',
    '04-11',
    '04-12',
    '04-12_1',
    '04-13',
    '04-14',
    '04-15',
]


In [17]:
def _valid_column(df, column):
    if column in _EXCLUDE_COLUMNS:
        return False

    column_prefix, var_name, postfix = COMBINE_harmonizer.flatten_column_tuple(column)
    if column_prefix not in _INCLUDE_COLUMN_PREFIX:
        return

    if 'Bayley' in column:
        return False

    if 'death' in column:
        return False

    if 'wdrawSupport' in column:
        return False

    if 'limitCare' in column:
        return False

    if re.search(r':followupCenter$', column):
        return False

    # XXX remove Date
    if re.search(r'Date\d*$', column):
        return False

    # XXX remove Time, but retain spontaneousRespirationTime
    if re.search(r'Time\d*$', column) and 'spontaneousRespirationTime' not in column:
        return False

    # with only MRINRNPatternOfInjury in MRI.
    if column_prefix == '03-05':
        if var_name != 'MRINRNPatternOfInjury':
            return False

    if df[column].dtype != np.float64:
        print(f'[INFO] invalid dtype: column: {column} dtype: {df[column].dtype}')
        return False

    if (df[column].isnull() == False).sum() < 20:
        print(f'[INFO] too few valid samples: column: {column} count: {(df[column].isnull() == False).sum()}')
        return False

    if (df[column].sem() == 0):
        print(f'[INFO] constant column: {column}')
        return False

    return True

In [18]:
x_columns = list(filter(lambda x: _valid_column(df_numeric, x), df_numeric.columns))
len(x_columns)

[INFO] invalid dtype: column: 01-02:siteID dtype: object
[INFO] invalid dtype: column: 01-02:screenComment dtype: object
[INFO] constant column: 01-02:coreTempLess32p5CGreaterEq2Hr_e
[INFO] constant column: 01-02:coreTempLess33p5CGreater1Hr_e
[INFO] constant column: 01-02:coreTempLess34CGreater1Hr_e
[INFO] constant column: 01-02:first6HrCoolByClinicalProtocol_e
[INFO] constant column: 01-02:chromosomalAbnormality_e
[INFO] constant column: 01-02:majorCongenitalAnomaly_e
[INFO] constant column: 01-02:birthWeightLessEq1800g_e
[INFO] constant column: 01-02:infantUnlikelySurvive_e
[INFO] constant column: 01-02:first60MinAllBloodGasPHGreater7p15BaseDeficitLess10mEqPerL_e
[INFO] constant column: 01-02:postnatalAgeLess6HrOrGreater24Hr_e
[INFO] constant column: 01-02:enrolledConflictingTrial_e
[INFO] constant column: 01-02:historyPerinatalEvent_i
[INFO] constant column: 01-02:at10MinApgarLessEq5OrVent_i
[INFO] constant column: 01-02:randomEligible
[INFO] invalid dtype: column: 01-02:consentStat

336

In [19]:
x_columns

['01-02:birthNumber',
 '01-02:first60MinAnyBloodGasPHLessEq7_i',
 '01-02:first60MinAnyBloodGasBaseDeficitGreaterEq16mEqPerL_i',
 '01-02:ageRand_hr',
 '01-02:treatmentBlanketType',
 '01-02:treatmentAssignmentDuration_hr',
 '01-02:treatmentAssignmentTemperature',
 '01-02:usualCoolingTreatmentGroup',
 '01-02:inOtherTrial',
 '01-03:motherAge_year',
 '01-03:motherRace2',
 '01-03:motherEducation',
 '01-03:motherEducation2',
 '01-03:motherInsurance',
 '01-03:motherInsurancePublic',
 '01-04:gravida',
 '01-04:parity',
 '01-04:multipleBirth',
 '01-04:prenatalCare',
 '01-04:hypertensionEclampsia',
 '01-04:antepartumHemorrhage',
 '01-04:thyroidMalfunction',
 '01-04:diabetes',
 '01-05:ruptureGreater18Hr',
 '01-05:ruptureBeforeDelivery',
 '01-05:labor',
 '01-05:fetalDecelerate',
 '01-05:cordMishap',
 '01-05:uterineRupture',
 '01-05:shoulderDystocia',
 '01-05:placentalProblem',
 '01-05:maternalHemorrhage',
 '01-05:maternalTrauma',
 '01-05:maternalCardioRespiratoryArrest',
 '01-05:maternalSeizure',
 '

In [20]:
df_corr_all = COMBINE_harmonizer.corr(df_numeric, x_columns, y_columns)

df_corr_all_pivot = pd.pivot_table(df_corr_all, index='x', columns='y', values=value_columns).transpose().swaplevel().transpose()

df_corr_all_pivot[('pvalue_lt_threshold', '')] = df_corr_all_pivot[('20-04:BayleyIIICognitiveComposite', 'pvalue')] < COMBINE_harmonizer.P_VALUE_THRESHOLD

all_reorder_columns = [('pvalue_lt_threshold', '')] + reorder_columns
df_corr_all_pivot = df_corr_all_pivot[all_reorder_columns].sort_values(by=[('pvalue_lt_threshold', ''), ('20-04:BayleyIIICognitiveComposite', 'r2')], ascending=[False, False]).reset_index(drop=False)
df_corr_all_pivot

out_filename = f'{out_dir}/31-02-corr-all.csv'
df_corr_all_pivot.to_csv(out_filename, index=False)

  0%|          | 0/336 [00:00<?, ?it/s]

_corr_coeff: to eval: x: 01-02:birthNumber y: 20-10:moderateSevereDisabilityOrDeath valid_x: 532 valid_y: 504
_corr_coeff: done: x: 01-02:birthNumber y: 20-10:moderateSevereDisabilityOrDeath


_corr_coeff: to eval: x: 01-02:birthNumber y: 20-10:disabilityLevelDeath4Category valid_x: 532 valid_y: 498
_corr_coeff: done: x: 01-02:birthNumber y: 20-10:disabilityLevelDeath4Category
_corr_coeff: to eval: x: 01-02:birthNumber y: 20-10_1:disabilityLevelDeath valid_x: 532 valid_y: 498
_corr_coeff: done: x: 01-02:birthNumber y: 20-10_1:disabilityLevelDeath
_corr_coeff: to eval: x: 01-02:birthNumber y: 20-04:BayleyIIICognitiveComposite valid_x: 532 valid_y: 421
_corr_coeff: done: x: 01-02:birthNumber y: 20-04:BayleyIIICognitiveComposite


  0%|          | 1/336 [00:00<01:10,  4.74it/s]

_corr_coeff: to eval: x: 01-02:birthNumber y: 20-04:BayleyIIILanguageComposite valid_x: 532 valid_y: 411
_corr_coeff: done: x: 01-02:birthNumber y: 20-04:BayleyIIILanguageComposite
_corr_coeff: to eval: x: 01-02:birthNumber y: 20-04:BayleyIIIMotorComposite valid_x: 532 valid_y: 415
_corr_coeff: done: x: 01-02:birthNumber y: 20-04:BayleyIIIMotorComposite


...

100%|█████████▉| 335/336 [01:11<00:00,  5.41it/s]

_corr_coeff: to eval: x: 04-15:dischargeHomeTherapyAnticonvulsantMedication y: 20-10_1:disabilityLevelDeath valid_x: 258 valid_y: 498
_corr_coeff: done: x: 04-15:dischargeHomeTherapyAnticonvulsantMedication y: 20-10_1:disabilityLevelDeath
_corr_coeff: to eval: x: 04-15:dischargeHomeTherapyAnticonvulsantMedication y: 20-04:BayleyIIICognitiveComposite valid_x: 258 valid_y: 421
_corr_coeff: done: x: 04-15:dischargeHomeTherapyAnticonvulsantMedication y: 20-04:BayleyIIICognitiveComposite
_corr_coeff: to eval: x: 04-15:dischargeHomeTherapyAnticonvulsantMedication y: 20-04:BayleyIIILanguageComposite valid_x: 258 valid_y: 411
_corr_coeff: done: x: 04-15:dischargeHomeTherapyAnticonvulsantMedication y: 20-04:BayleyIIILanguageComposite
_corr_coeff: to eval: x: 04-15:dischargeHomeTherapyAnticonvulsantMedication y: 20-04:BayleyIIIMotorComposite valid_x: 258 valid_y: 415
_corr_coeff: done: x: 04-15:dischargeHomeTherapyAnticonvulsantMedication y: 20-04:BayleyIIIMotorComposite
_corr_coeff: to eval: x:

100%|██████████| 336/336 [01:11<00:00,  5.06it/s]

100%|██████████| 336/336 [01:11<00:00,  4.68it/s]

_corr_coeff: to eval: x: 04-15:dischargeHomeTherapyOther y: 20-10_1:disabilityLevelDeath valid_x: 258 valid_y: 498
_corr_coeff: done: x: 04-15:dischargeHomeTherapyOther y: 20-10_1:disabilityLevelDeath
_corr_coeff: to eval: x: 04-15:dischargeHomeTherapyOther y: 20-04:BayleyIIICognitiveComposite valid_x: 258 valid_y: 421
_corr_coeff: done: x: 04-15:dischargeHomeTherapyOther y: 20-04:BayleyIIICognitiveComposite
_corr_coeff: to eval: x: 04-15:dischargeHomeTherapyOther y: 20-04:BayleyIIILanguageComposite valid_x: 258 valid_y: 411
_corr_coeff: done: x: 04-15:dischargeHomeTherapyOther y: 20-04:BayleyIIILanguageComposite
_corr_coeff: to eval: x: 04-15:dischargeHomeTherapyOther y: 20-04:BayleyIIIMotorComposite valid_x: 258 valid_y: 415
_corr_coeff: done: x: 04-15:dischargeHomeTherapyOther y: 20-04:BayleyIIIMotorComposite


In [21]:
df_corr_all_pivot

y                                        x pvalue_lt_threshold  \
                                                                 
0                        03-05_s:MRIExtent                True   
1        03-05_s:MRIWhiteMatterInjuryMerge                True   
2                      03-05_s:MRIBGTMerge                True   
3            03-05:MRINRNPatternOfInjury:1                True   
4                     03-05_s:MRIPLICMerge                True   
..                                     ...                 ...   
331                01-06:at10MinIntubation               False   
332                     04-04:dischargeDIC               False   
333            01-06:at10MinBaggingAndMask               False   
334  04-13:dischargeSeizurePreIntervention               False   
335                04-06:dischargeDialysis               False   

y   20-10:moderateSevereDisabilityOrDeath                                 \
                                     corr        r2        pvalue  valid   
0                                0.698431  0.487806  4.796415e-26  169.0   
1                                0.715021  0.511255  1.960550e-19  116.0   
2                                0.649612  0.421995  4.141362e-37  298.0   
3                                0.549684  0.302152  5.320614e-28  337.0   
4                                0.597946  0.357539  2.805184e-30  298.0   
..                                    ...       ...           ...    ...   
331                              0.145105  0.021055  3.269784e-03  409.0   
332                              0.228704  0.052306  2.096174e-07  504.0   
333                              0.016057  0.000258  7.458231e-01  410.0   
334                             -0.074098  0.005490  2.083478e-01  290.0   
335                              0.068173  0.004648  1.263966e-01  504.0   

y                                 20-10:disabilityLevelDeath4Category  ...  \
    valid_x valid_y valid_percent                                corr  ...   
0     169.0   504.0      0.317669                            0.687614  ...   
1     116.0   504.0      0.218045                            0.706771  ...   
2     298.0   504.0      0.560150                            0.650111  ...   
3     355.0   504.0      0.633459                            0.551901  ...   
4     298.0   504.0      0.560150                            0.596209  ...   
..      ...     ...           ...                                 ...  ...   
331   429.0   504.0      0.768797                            0.136233  ...   
332   532.0   504.0      0.947368                            0.221352  ...   
333   430.0   504.0      0.770677                            0.005861  ...   
334   305.0   504.0      0.545113                           -0.087448  ...   
335   532.0   504.0      0.947368                            0.066167  ...   

y   20-04:BayleyIIILanguageComposite                        \
                             valid_x valid_y valid_percent   
0                              169.0   411.0      0.263158   
1                              116.0   411.0      0.172932   
2                              298.0   411.0      0.494361   
3                              355.0   411.0      0.554511   
4                              298.0   411.0      0.494361   
..                               ...     ...           ...   
331                            429.0   411.0      0.612782   
332                            532.0   411.0      0.772556   
333                            430.0   411.0      0.612782   
334                            305.0   411.0      0.421053   
335                            532.0   411.0      0.772556   

y   20-04:BayleyIIIMotorComposite                                         \
                             corr        r2        pvalue  valid valid_x   
0                       -0.673434  0.453514  3.118979e-20  143.0   169.0   
1                       -0.636655  0.405329  4.073914e-12   95.0   116.0   
2                       -0

## 04. Plot

In [22]:
def _plot_corr(df, x_column, y_column, x_column_info=None, y_column_info=None, is_close=False):
    fig = COMBINE_harmonizer.plot_corr(df, x_column, y_column, x_column_info, y_column_info)

    x_str = re.sub(r':', '_', x_column)
    y_str = re.sub(r':', '_', y_column)

    fig.savefig(f'{out_figure_dir}/31-{x_str}-{y_str}-corr.pdf', bbox_inches='tight')
    fig.savefig(f'{out_figure_dir}/31-{x_str}-{y_str}-corr.png', bbox_inches='tight')
    fig.savefig(f'{out_figure_dir}/31-{x_str}-{y_str}-corr.eps', bbox_inches='tight')
    if is_close:
        plt.close(fig)


In [23]:
y_columns_map = {
    '20-04:BayleyIIICognitiveComposite': {'title': 'Bayley III Cognitive Composite Score', 'unit': ''},
    # '20-04:BayleyIIILanguageComposite': {'title': 'Bayley III Language Composite Score', 'unit': ''},
    # '20-04:BayleyIIIMotorComposite': {'title': 'Bayley III Motor Composite Score', 'unit': ''},

    # '30-02:moderateSevereDisabilityOrDeath': {'title': 'Death or Moderate/Severe Disability', 'unit': ''},

    # '30-02:disabilityLevel': {'title': 'Disability Level', 'unit': ''},

    '20-10_1:disabilityLevelDeath': {'title': 'Disability Level', 'unit': ''},
}

x_columns_map = {
    # '01-06:firstPostnatalBloodGasPH': {'title': 'Blood Gas pH within 1st HR', 'unit': ''},

    # '04-04:dischargeChronicLungDisease': {'title': 'Discharge - Chronic Lung Disease', 'unit': ''},

    # '20-09:gastrostomyTube_b': {'title': 'Gastrostomy Tube', 'unit': ''},

    # '04-08:dischargeFullNippleFeed': {'title': 'Discharge - Full Nipple Feed', 'unit': ''},

    # '01-06:Apgar5min': {'title': 'Apgar Score at 5 min', 'unit': ''},

    # '01-06:Apgar10min': {'title': 'Apgar Score at 10 min', 'unit': ''},

    # '01-06:encephalopathyLevel': {'title': 'Encephalophathy Level', 'unit': ''},

    # '01-06:initBloodGasPH': {'title': 'Blood Gas pH', 'unit': ''},

    # '01-06:cordBloodGasPH': {'title': 'Cord Blood Gas pH', 'unit': ''},

    # '03-05_s1:MRINRNPatternOfInjuryMerge': {'title': 'NRN Pattern of Injury', 'unit': ''},
    # '03-05_s1:MRINRNPatternOfInjuryAvg': {'title': 'NRN Pattern of Injury - Mean', 'unit': ''},
    # '03-05_s1:MRINRNPatternOfInjuryMax': {'title': 'NRN Pattern of Injury - Max', 'unit': ''},

    # '03-05_s:MRI2LevelPatternOfInjury': {'title': 'NRN Pattern of Injury - 2 Level', 'unit': ''},
    # '03-05_s:MRINRNPatternOfInjuryMerge': {'title': 'NRN Pattern of Injury', 'unit': ''},

    # '01-12_1:pre_TotalModifiedSarnatScore': {'title': 'Total Modified Sarnat Score - Screening', 'unit': ''},
    # '03-04_1:post_TotalModifiedSarnatScore': {'title': 'Total Modified Sarnat Score - Post-treatment', 'unit': ''},
    # '04-12_1:dischargeTotalModifiedSarnatScore': {'title': 'Total Modified Sarnat Score - Discharge', 'unit': ''},

    # '01-12:pre_NeuroExamSeizure': {'title': 'Seizure - Screening', 'unit': ''},

    # '01-06:Apgar10minLte5': {'title': 'Apgar Score at 10 min $\leq$ 5', 'unit': ''},

    # '01-06:Apgar10minLt5': {'title': 'Apgar Score at 10 min < 5', 'unit': ''},

    # '01-06:infantSex': {'title': 'Infant Sex', 'unit': ''},
    # '01-06:maleSex': {'title': 'Infant Sex', 'unit': ''},

    # '01-06:birthGestationalAge_week': {'title': 'Birth Gestational Age', 'unit': 'Week'},
    # '01-03:motherEducation': {'title': 'Mother Education', 'unit': ''},
    # '01-03:motherAge_year': {'title': 'Mother Age', 'unit': 'Year'},

    # '01-06:birthWeight_g': {'title': 'Birth Weight', 'unit': 'g'},

    # '04-01_1:lengthOfStay_day': {'title': 'Length of Stay', 'unit': 'Day'},
}

for x_column, x_column_info in x_columns_map.items():
    for y_column, y_column_info in y_columns_map.items():
        _plot_corr(df_numeric, x_column, y_column, x_column_info, y_column_info, is_close=True)